# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#get info from csv in weatherpy
cities_df = pd.read_csv("../weatherpy/output_data/city_weather.csv")
cities_df.head()

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,hobart,41.00,86,147.33,-42.88,75,AU,6.93,1602878272
1,1,nikolskoye,41.00,65,30.79,59.70,90,RU,6.71,1602878273
2,2,coaticook,48.20,100,-71.80,45.13,90,CA,4.70,1602878274
3,3,yumen,41.63,50,97.20,40.28,4,CN,13.51,1602878276
4,4,college,17.01,72,-147.80,64.86,20,US,5.82,1602878277


### Humidity Heatmap

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Store latitude, longitude and Humidity in locations
locations = cities_df[["Latitude", "Longitude"]]
Humidity = cities_df["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#create new data with specific conditions
ideal_cities = cities_df.loc[(cities_df["Temperature"]>=70)\
                             &(cities_df["Temperature"]<=80)\
                             &(cities_df["Windspeed"]<=10)\
                             &(cities_df["Cloudiness"]<=0)\
                             &(cities_df["Humidity"]<=50)].dropna()
ideal_cities

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
58,58,kutum,79.95,17,24.67,14.20,0,SD,9.64,1602878349
109,109,behbahan,71.60,18,50.24,30.60,0,IR,1.63,1602878425
173,173,buin,78.01,41,-70.75,-33.73,0,CL,8.05,1602878512
316,316,marzuq,70.20,23,46.47,14.40,0,YE,2.89,1602878690
379,379,sakakah,75.20,28,40.21,29.97,0,SA,3.36,1602878771
383,383,letlhakeng,79.16,21,25.03,-24.09,0,BW,5.86,1602878775
429,429,sur,78.42,50,59.53,22.57,0,OM,9.51,1602878720
556,556,kwekwe,70.07,37,29.81,-18.93,0,ZW,10.00,1602878994
569,569,alice springs,75.00,42,133.88,-23.70,0,AU,9.08,1602878801


### Hotel Map

In [10]:
#create new df *copy to avoid the error message
hotel_df = ideal_cities[["City","Country","Latitude", "Longitude"]].copy()
#add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
58,kutum,SD,14.20,24.67,
109,behbahan,IR,30.60,50.24,
173,buin,CL,-33.73,-70.75,
316,marzuq,YE,14.40,46.47,
379,sakakah,SA,29.97,40.21,
383,letlhakeng,BW,-24.09,25.03,
429,sur,OM,22.57,59.53,
556,kwekwe,ZW,-18.93,29.81,
569,alice springs,AU,-23.70,133.88,


In [11]:
# find the closest hotels with params using gkey
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    latitude = row['Latitude']
    longitude = row['Longitude']

    # add keyword to params dict
    params['location'] = f'{latitude},{longitude}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {row['City']} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 58: kutum.
Closest kutum hotel is Dibbo areaمنطقة دبو.
------------
Retrieving Results for Index 109: behbahan.
Closest behbahan hotel is Esteghlal Hotel.
------------
Retrieving Results for Index 173: buin.
Closest buin hotel is Paihuen.
------------
Retrieving Results for Index 316: marzuq.
Missing field/result... skipping.
------------
Retrieving Results for Index 379: sakakah.
Closest sakakah hotel is Fakhamat Al Orjoana Apartment.
------------
Retrieving Results for Index 383: letlhakeng.
Closest letlhakeng hotel is Tokey's Lodge.
------------
Retrieving Results for Index 429: sur.
Closest sur hotel is Sur Plaza Hotel.
------------
Retrieving Results for Index 556: kwekwe.
Closest kwekwe hotel is Golden Mile Hotel, Kwekwe.
------------
Retrieving Results for Index 569: alice springs.
Closest alice springs hotel is Desert Palms Alice Springs.
------------


In [12]:
#df with set columns and name of hotels
hotel_df.head(10)

,City,Country,Latitude,Longitude,Hotel Name
58,kutum,SD,14.20,24.67,Dibbo areaمنطقة دبو
109,behbahan,IR,30.60,50.24,Esteghlal Hotel
173,buin,CL,-33.73,-70.75,Paihuen
316,marzuq,YE,14.40,46.47,
379,sakakah,SA,29.97,40.21,Fakhamat Al Orjoana Apartment
383,letlhakeng,BW,-24.09,25.03,Tokey's Lodge
429,sur,OM,22.57,59.53,Sur Plaza Hotel
556,kwekwe,ZW,-18.93,29.81,"Golden Mile Hotel, Kwekwe"
569,alice springs,AU,-23.70,133.88,Desert Palms Alice Springs


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))